# <b>Price Predictor

### First we'll do a year of one company:

In [243]:
import numpy as np
import pandas as pd
import matplotlib
import yfinance as yf

In [244]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S%26P_500_component_stocks')
table_symbol = data[0]
table_industry = data[0]['GICS Sector']

#these 2 lists are in order with one another
symbols = list(table_symbol.Symbol.values)
industries = list(table_industry.values) ##Industry Sector


remove = []

symbols_to_remove = ['BF.B', 'BRK.B', 'KVUE', 'VLTO','SPY']

for i, symbol in enumerate(symbols):
    if symbol in symbols_to_remove:
        remove.append(i)
#print(remove)

#list(set(industries))

for i in remove:
    symbols.remove(symbols[i])
    industries.remove(industries[i])

print(len(symbols))

symbol_industry = dict(zip(symbols,industries))
print(len(symbol_industry))

499
499


In [245]:
tickers = yf.Tickers('AAPL')

APPLdf = tickers.tickers['AAPL'].history(period="5d", start="2022-01-01", end="2022-12-31")
APPLdf.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,175.821100,180.814055,175.702461,179.953873,104487900,0.0,0.0
2022-01-04 00:00:00-05:00,180.566907,180.873403,177.096548,177.669998,99310400,0.0,0.0
2022-01-05 00:00:00-05:00,177.581006,178.134677,172.667149,172.943985,94537600,0.0,0.0
2022-01-06 00:00:00-05:00,170.749066,173.319700,169.701043,170.056976,96904000,0.0,0.0
2022-01-07 00:00:00-05:00,170.936919,172.172799,169.097931,170.225052,86709100,0.0,0.0
2022-01-10 00:00:00-05:00,167.169959,170.551323,166.270236,170.244827,106765600,0.0,0.0
2022-01-11 00:00:00-05:00,170.373372,173.201049,168.890317,173.102188,76138300,0.0,0.0
2022-01-12 00:00:00-05:00,174.130420,175.178443,172.845118,173.547089,74805200,0.0,0.0
2022-01-13 00:00:00-05:00,173.794269,174.624776,169.849337,170.244827,84505800,0.0,0.0


In [246]:
drop_cols = ['Open', 'Low', 'High','Volume', 'Dividends', 'Stock Splits']
APPLdf = APPLdf.drop(columns=drop_cols, axis = 1)
APPLdf.index = APPLdf.index.astype(str).str.split(' ').str[0]
APPLdf

,Close
Date,
2022-01-03,179.953873
2022-01-04,177.669998
2022-01-05,172.943985
2022-01-06,170.056976
2022-01-07,170.225052
...,...
2022-12-23,131.127060
2022-12-27,129.307236
2022-12-28,125.339417


In [247]:
APPLdf.loc["2022-06-15"]

Close    134.267685
Name: 2022-06-15, dtype: float64

In [248]:
APPLdf.loc["2022-11-25"]
#APPL.loc["2022-11-24"]

Close    147.286743
Name: 2022-11-25, dtype: float64

In [249]:
holidays = ["2022-01-17","2022-02-21","2022-04-15", "2022-05-30", "2022-06-20", "2022-07-4", "2022-09-5", "2022-11-24","2022-12-26"]
len(holidays)

9

In [250]:
holiday_df = pd.DataFrame(index=holidays)
holiday_df['Date'] = holiday_df.index

# Add another column 'Close' with NaN values
holiday_df['Close'] = float('nan')

# Set the 'Date' column as the index
holiday_df.set_index('Date', inplace=True)

# Print the resulting DataFrame
holiday_df

,Close
Date,
2022-01-17,NaN
2022-02-21,NaN
2022-04-15,NaN
2022-05-30,NaN
2022-06-20,NaN
2022-07-4,NaN
2022-09-5,NaN
2022-11-24,NaN
2022-12-26,NaN


In [251]:
# Concatenate the original DataFrame with the holiday DataFrame
frames = [APPLdf, holiday_df]
result_df = pd.concat(frames)

# Sort the index to maintain order
result_df = result_df.sort_index()

# Print the resulting DataFrame
APPL1 = result_df
APPL1
#APPL1 has NaN values
#APPLdf has no NaN values

,Close
Date,
2022-01-03,179.953873
2022-01-04,177.669998
2022-01-05,172.943985
2022-01-06,170.056976
2022-01-07,170.225052
...,...
2022-12-26,NaN
2022-12-27,129.307236
2022-12-28,125.339417


## <b>APPL data for the year:

In [252]:
# Convert index to datetime if it's in string format
APPL1.index = pd.to_datetime(APPL1.index)

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(index=[f'APPL Week {week}' for week in range(1, 53)],
                      columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through rows of the original DataFrame and fill the new DataFrame
for i, row in APPL1.iterrows():
    week = i.week
    day = i.dayofweek
    
    # If there are rows for the current week and day, set the mean close price
    if not pd.isna(row['Close']):
        new_df.at[f'APPL Week {week}', f'Day {day + 1}'] = row['Close']

# Display the new DataFrame
APPL = new_df
APPL


,Day 1,Day 2,Day 3,Day 4,Day 5
APPL Week 1,179.953873,177.669998,172.943985,170.056976,170.225052
APPL Week 2,170.244827,173.102188,173.547089,170.244827,171.114914
APPL Week 3,NaN,167.881836,164.352142,162.651611,160.575317
APPL Week 4,159.79422,157.975006,157.886032,157.421326,168.405838
APPL Week 5,172.805557,172.637512,173.853561,170.946808,170.659714
APPL Week 6,169.937057,173.075226,174.510651,170.392426,166.947357
APPL Week 7,167.184937,171.055725,170.81813,167.184937,165.620819
APPL Week 8,NaN,162.670746,158.463379,161.106583,163.195389
APPL Week 9,163.462677,161.561966,164.88826,164.561539,161.532257
APPL Week 10,157.701141,155.859772,161.314484,156.92894,153.176971


In [253]:
APPL1.head(20)

,Close
Date,
2022-01-03,179.953873
2022-01-04,177.669998
2022-01-05,172.943985
2022-01-06,170.056976
2022-01-07,170.225052
2022-01-10,170.244827
2022-01-11,173.102188
2022-01-12,173.547089
2022-01-13,170.244827


## <b>Week 1 of all the stocks

In [254]:
#symbols = list of stocks
# Create an empty DataFrame to store the results
# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through each symbol
for symbol in symbols:
    # Fetch historical stock data for the symbol
    ticker = yf.Ticker(symbol)
    stock_data = ticker.history(period="5d", start="2022-01-03", end= "2022-01-10")

    # Extract the close prices for the first week
    close_prices = stock_data['Close'].tolist()[:5]

    # Create a temporary DataFrame for the current symbol
    temp_df = pd.DataFrame([close_prices], columns=[f'Day {i+1}' for i in range(len(close_prices))],
                           index=[f'{symbol} Week 1'])

    # Concatenate the temporary DataFrame with the result DataFrame
    result_df = pd.concat([result_df, temp_df])

# Display the resulting DataFrame
print(result_df)


/var/folders/bq/q6bbfq_s68l_m1x6yxgvqnt80000gn/T/ipykernel_23146/4155590832.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, temp_df])
BF.B: Period '5d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
CEG: Data doesn't exist for startDate = 1641186000, endDate = 1641790800
GEHC: Data doesn't exist for startDate = 1641186000, endDate = 1641790800
KVUE: Data doesn't exist for startDate = 1641186000, endDate = 1641790800
VLTO: Data doesn't exist for startDate = 1641186000, endDate = 1641790800


                  Day 1       Day 2       Day 3       Day 4       Day 5
MMM Week 1   163.319931  165.607925  164.927963  163.558838  165.350662
AOS Week 1    80.547264   81.886978   80.518333   81.058098   80.142441
ABT Week 1   134.063416  130.910477  130.322311  130.303009  130.707993
ABBV Week 1  125.280724  125.040176  125.697006  125.104935  124.781136
ACN Week 1   395.446655  392.620728  385.706390  367.080475  360.039886
...                 ...         ...         ...         ...         ...
YUM Week 1   132.103195  132.809540  131.125961  132.480576  130.961472
ZBRA Week 1  583.900024  587.599976  558.179993  555.159973  530.859985
ZBH Week 1   123.620804  124.874924  124.463257  123.429329  122.682617
ZION Week 1   59.537762   61.233822   61.372841   63.893742   64.959564
ZTS Week 1   230.225357  221.460205  213.039368  213.885391  207.658279

[499 rows x 5 columns]


In [262]:
result_df

,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 1,163.319931,165.607925,164.927963,163.558838,165.350662
AOS Week 1,80.547264,81.886978,80.518333,81.058098,80.142441
ABT Week 1,134.063416,130.910477,130.322311,130.303009,130.707993
ABBV Week 1,125.280724,125.040176,125.697006,125.104935,124.781136
ACN Week 1,395.446655,392.620728,385.706390,367.080475,360.039886
...,...,...,...,...,...
YUM Week 1,132.103195,132.809540,131.125961,132.480576,130.961472
ZBRA Week 1,583.900024,587.599976,558.179993,555.159973,530.859985
ZBH Week 1,123.620804,124.874924,124.463257,123.429329,122.682617
ZION Week 1,59.537762,61.233822,61.372841,63.893742,64.959564


In [255]:
result_df.shape

(499, 5)

In [258]:
result_df.loc["CEG Week 1"]

Day 1   NaN
Day 2   NaN
Day 3   NaN
Day 4   NaN
Day 5   NaN
Name: CEG Week 1, dtype: float64

In [267]:
rows_with_nan = result_df[result_df.isna().any(axis=1)].index.tolist()
rows_with_nan

['BF.B Week 1', 'CEG Week 1', 'GEHC Week 1', 'KVUE Week 1', 'VLTO Week 1']

In [268]:
#drop them
temp = result_df.T.drop(columns=rows_with_nan, axis = 1)
week1 = temp.T

In [269]:
week1

,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 1,163.319931,165.607925,164.927963,163.558838,165.350662
AOS Week 1,80.547264,81.886978,80.518333,81.058098,80.142441
ABT Week 1,134.063416,130.910477,130.322311,130.303009,130.707993
ABBV Week 1,125.280724,125.040176,125.697006,125.104935,124.781136
ACN Week 1,395.446655,392.620728,385.706390,367.080475,360.039886
...,...,...,...,...,...
YUM Week 1,132.103195,132.809540,131.125961,132.480576,130.961472
ZBRA Week 1,583.900024,587.599976,558.179993,555.159973,530.859985
ZBH Week 1,123.620804,124.874924,124.463257,123.429329,122.682617
ZION Week 1,59.537762,61.233822,61.372841,63.893742,64.959564


In [270]:
week1.isna().any()

Day 1    False
Day 2    False
Day 3    False
Day 4    False
Day 5    False
dtype: bool

In [272]:
all = pd.DataFrame(columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through each symbol
for symbol in symbols:
    # Fetch historical stock data for the symbol
    ticker = yf.Ticker(symbol)
    stock_data = ticker.history(period="1y")  # Fetch data for the last 1 year

    # Extract the close prices for each week
    for week in range(1, 53):
        start_date = f"2022-01-01"  # Change the start date as needed
        end_date = f"2022-12-31"    # Change the end date as needed
        week_data = stock_data[start_date:end_date]

        # If data for the current week is missing, create a row with NaN values
        if len(week_data) < 5:
            week_row = pd.DataFrame([[np.nan] * 5], columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'],
                                     index=[f'{symbol} Week {week}'])
        else:
            week_row = pd.DataFrame([week_data['Close'].tolist()[:5]], columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'],
                                     index=[f'{symbol} Week {week}'])

        # Concatenate the temporary DataFrame with the result DataFrame
        all = pd.concat([all, week_row])

# Display the resulting DataFrame
all

/var/folders/bq/q6bbfq_s68l_m1x6yxgvqnt80000gn/T/ipykernel_23146/2891734177.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all = pd.concat([all, week_row])
BF.B: No price data found, symbol may be delisted (period=1y)


,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 1,125.898163,124.052010,124.231888,121.883972,121.788185
MMM Week 2,125.898163,124.052010,124.231888,121.883972,121.788185
MMM Week 3,125.898163,124.052010,124.231888,121.883972,121.788185
MMM Week 4,125.898163,124.052010,124.231888,121.883972,121.788185
MMM Week 5,125.898163,124.052010,124.231888,121.883972,121.788185
...,...,...,...,...,...
ZTS Week 48,147.207199,145.126175,148.465729,146.939636,142.301941
ZTS Week 49,147.207199,145.126175,148.465729,146.939636,142.301941
ZTS Week 50,147.207199,145.126175,148.465729,146.939636,142.301941
ZTS Week 51,147.207199,145.126175,148.465729,146.939636,142.301941


In [274]:
naners = all[all.isna().any(axis=1)].index.tolist()
print(len(naners))

728


In [275]:
#drop them
temp = all.T.drop(columns=naners, axis = 1)
sp500_52weeks = temp.T

In [276]:
sp500_52weeks

,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 1,125.898163,124.052010,124.231888,121.883972,121.788185
MMM Week 2,125.898163,124.052010,124.231888,121.883972,121.788185
MMM Week 3,125.898163,124.052010,124.231888,121.883972,121.788185
MMM Week 4,125.898163,124.052010,124.231888,121.883972,121.788185
MMM Week 5,125.898163,124.052010,124.231888,121.883972,121.788185
...,...,...,...,...,...
ZTS Week 48,147.207199,145.126175,148.465729,146.939636,142.301941
ZTS Week 49,147.207199,145.126175,148.465729,146.939636,142.301941
ZTS Week 50,147.207199,145.126175,148.465729,146.939636,142.301941
ZTS Week 51,147.207199,145.126175,148.465729,146.939636,142.301941


In [277]:
sp500_52weeks.isna().any()

Day 1    False
Day 2    False
Day 3    False
Day 4    False
Day 5    False
dtype: bool

In [278]:
sp500_pct = sp500_52weeks.pct_change()
sp500_pct

,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 1,NaN,NaN,NaN,NaN,NaN
MMM Week 2,0.0,0.0,0.0,0.0,0.0
MMM Week 3,0.0,0.0,0.0,0.0,0.0
MMM Week 4,0.0,0.0,0.0,0.0,0.0
MMM Week 5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
ZTS Week 48,0.0,0.0,0.0,0.0,0.0
ZTS Week 49,0.0,0.0,0.0,0.0,0.0
ZTS Week 50,0.0,0.0,0.0,0.0,0.0
ZTS Week 51,0.0,0.0,0.0,0.0,0.0
